<h1 style="font-size: 40px; margin-bottom: 0px;">6.1 Intro to ChIP-seq pipeline</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

We're now moving on to the ChIP-seq analysis module of this course. The goal of this module is to use big data analysis to identify the genes that may be regulated by TAZ by looking at the regions of DNA to which TAZ (and its associated TEAD DNA-binding protein) actually bind. In class, we'll practice using a truncated dataset to allow us to perform the analyses on a small subset of the larger dataset, and outside of class (aka for homework assignments), you will run the analysis using the full dataset. If there's time in class, you can also start running analyses on the complete dataset to save some time. The full dataset is rather large, so the analysis will take some time to run. You will want to keep this in mind, since your assignments will require you to turn in the results of your analysis.
    
To start, we'll go over the basic concepts that underlie ChIP-seq, and then we'll have a small command line refresher and also learn more about command line syntax to prepare ourselves to work with our sequencing files. We'll then run a QC to understand the quality of our sequencing data.

<strong>Learning objectives:</strong>
<ul>
    <li>Understand concepts behind ChIP-seq</li>
    <li>Review command line/Terminal</li>
    <li>Learn more about how to work in Terminal</li>
    <li>Learn to make sense of your environment</li>
    <li>Learn to perform QC on sequencing data</li>
    <li>Understand how to interpret QC results</li>
</ul>

<h1 style="font-size: 40px; margin-bottom: 0px;">ChIP-seq overview</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<h2>Experimental design</h2>

One of the main concepts that underlies ChIP-seq is a technique called chromatin immunoprecipitation (ChIP). Like with the more general technique of immunoprecipitation, ChIP takes advantage of the specificity of antibodies in order to select for specific proteins, namely proteins that interact with DNA either directly (such as histones) or indirectly through interactions with a DNA-binding protein (Park 2009).

Prior to immunoprecipitation, samples are crosslinked with formaldehyde in order to stabilize molecular interactions, such as protein-protein or protein-DNA interactions (<strong>Fig 1</strong>). Then the DNA is fragmented via sonication, producing smaller fragments of DNA ranging from 200 to 600 base pairs, with any associated proteins still bound to it. DNA fragmentation is followed by immunoprecipitation using antibodies specific to the target protein or interest. As the target protein is pulled down, any bound DNA is pulled down along with it, allowing us to enrich for the regions in which the target protein is closely associated. To release the DNA from the protein, the crosslinking reaction is reversed, and the DNA can be purified. Afterwards, a sequencing library is prepared from the released DNA fragments, and next-generation sequencing (NGS) is carried out.

<h4 style="text-align: center;"><strong>Fig 1</strong></h4>
<img src="./images/6_1_fig_1.png" style="height: 450px; margin: auto;"/>
<p style="text-align: center;">Image from: Park 2009 Nat Rev Genet</p>

<h2>ChIP-seq analysis pipeline overview</h2>

Below is a brief overview of the main steps taken to analyze ChIP-seq data, and in later lectures, you'll learn about them in more detail.

<h3>Raw sequence file</h3>

As you'll recall from MCB201A, NGS allows for massively parallel sequencing of the DNA fragments in a library prep. The resulting sequence output contains all the sequence reads and the associated quality information for each base. The type of file it usually comes as is a compressed fastq (<code>.fastq.gz</code>) file. A <code>.fastq</code> file has a specific structure, where each sequencing read corresponds to 4 lines within the file, so a fastq file containing the sequence information for 10 reads will be 40 lines long.

Each line for a sequence read contains a specific set of information:

<table style="margin:auto;">
    <tr style="border: 1px solid; border-color: #000000;">
        <th>Line</th>
        <th>Details</th>
    </tr>
    <tr style="border: 1px solid; border-color: #000000;">
        <td>1</td>
        <td>A sequence identifier that contains information on the run</td>
    </tr>
    <tr style="border: 1px solid; border-color: #000000;">
        <td>2</td>
        <td>The sequence information</td>
    </tr>
    <tr style="border: 1px solid; border-color: #000000;">
        <td>3</td>
        <td>A separater line. Begins with <strong>&plus;</strong> and sometimes followed by the sequence identifier</td>
    </tr>
    <tr style="border: 1px solid; border-color: #000000;">
        <td>4</td>
        <td>Quality score information for each nucleotide encoded using ASCII to represent the numerical quality value</td>
    </tr>
</table>



How this looks like if you pull out the information for a single sequence read from a fastq file is something like:

```
@LH00328:659:235GCFLT3:5:1101:37786:1048 1:N:0:ATTCTGGC+TCTGGACA
CNTCCGGATGGCTTTGTTGATCCGCTGCACAGCGTGGAGCATGGCTTGTTCCTGATCACCCTTTGTGTTGGCTGCAGCCACACCAGCCTGGACTTCCTCCTTTTCCAGAAGCTCCACCAGGCCCAGCTCCTGTGCCTTCTGCTCTCTGTC
+
I#IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIII9IIIIIIIIIIIIIIIIIIIIIIIII
```

The quality score contained within the fourth line tells us the probability that the called base is incorrect. In other words, it tells us the likelihood that the sequence read is incorrect for that specific nucleotide. Using the example above, we can see that the first nucleotide <code>C</code> has an associated quality score encoded as <code>I</code>, which corresponds to a Phred-33 quality score of 40 (<a href="https://people.duke.edu/~ccc14/duke-hts-2018/bioinformatics/quality_scores.html#Phred-Quality-Scores" rel="noopener noreferrer" target="_blank"><u>More information on Phred quality scores can be found here</u></a>). This tells us that the probability of the first base being incorrectly sequenced is 0.00008.

<code>.fastq</code> files are often incredibly large, so they are usually compressed into <code>.fastq.gz</code> file. Most programs that analyze sequencing data are able to handle both <code>.fastq</code> and <code>.fastq.gz</code> files, so you don't need to unzip the .fastq.gz file to use them for analysis.

Let's jump into Terminal and pull out some sequence information from one of our truncated <code>.fastq</code> files.

For this, you have the option of following along either in your notebook, where you will run bash commands in a code cell or by following along in Terminal. 

In [1]:
%%bash
#Each cell where we want to run command line in this notebook will start with %%bash
#This opens up a temporary shell for us to run command line
#You'll want to keep in mind that you need to do this for each code cell,
#and each instance is separate, so if you change directories in one
#It won't be reflected in another

#We're pulling out the first four rows
#Each read in a fastq file has 4 rows of information
#So here, we're only pulling out a single read
head -4 ~/shared/2025-fall/courses/1547808/chip-seq/truncated/10M_ctrl_1.fastq

@SRR1810912.1 HWI-ST1210:136:C1RBDACXX:5:1101:1472:2121 length=50
CACTGTAATTTCCCTAAAACAGGAGGTCACGGTAAGTCCTATGGATAGAT
+SRR1810912.1 HWI-ST1210:136:C1RBDACXX:5:1101:1472:2121 length=50
??????????????????????????????????????????????????


<h3>Quality assessment</h3>

Prior to performing your analysis, you'll want to check the quality of your sequence reads in order to determine if there might be issues with your samples that you will want to keep in mind when performing your analyses. These issues could have arised from sample prep or the sequencing run itself, and they can point to biases or other confounding factors that can impact the interpretation of your sequencing data. This is the case for essentially anytime you're analyzing your NGS data, and we'll be doing this for our ChIP-seq dataset and later on for our RNA-seq dataset.
    
A common method to performing quality checks on raw sequence data is called <a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/" rel="noopener noreferrer" target="_blank"><u>FastQC</u></a>, developed by Babraham Bioinformatics. FastQC takes in <code>.fastq</code> (and the compressed form <code>.fastq.gz</code>) files and analyzes the raw sequence data with a series of modules that can highlight potential issues in a dataset. <a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/" rel="noopener noreferrer" target="_blank"><u>Information on the different modules and an explanation on how to interpret their results can be found in the FastQC documentation.</u></a>

After we run through the overview of ChIP-seq and our command line refresher/practice, we'll run a QC on our ChIP-seq data files using FastQC.

<h3>Genome alignment</h3>

To determine where our sequence reads came from, we need to align them to a reference genome (<strong>Fig 2</strong>), and the resulting alignments allow us to determine the distribution of sequence reads along the genome. In ChIP-seq, the resulting distribution of aligned sequences gives us an idea of the downstream transcriptional targets of a protein of interest. In our case, it will shed light on where TAZ (and TEAD) are binding and provide hints to what genes TAZ may be regulating, since we should be recovering a greater number of fragments corresponding to regions in the genome to which TAZ (and TEAD) are binding.

<h4 style="text-align: center;"><strong>Fig 2</strong></h4>
<img src="./images/6_1_fig_2.png" style="height: 250px; margin: auto;"/>
<p style="text-align: center;">Image from: Harvard Chan Bioinformatics Core</p>

<h3>Peak calling and visualization</h3>

Following genome alignment, regions of high alignment can be determined by peak calling (<strong>Fig 3</strong>). Tools such as MACS2 use the aligned reads to calculate the density of read counts throughout the genome in order to identify regions that have a large number of aligned reads. These peaks in read density tell us that the identified region is enriched within a dataset, suggesting a potential binding site for a protein of interest.

<h4 style="text-align: center;"><strong>Fig 3</strong></h4>
<img src="./images/6_1_fig_3.png" style="width: 400px; margin: auto;"/>
<p style="text-align: center;">Image from: <a href="#wilbanksfacciotti2010"><u>Wilbanks and Facciotti 2010 PLoS ONE</u></a></p>

After we call our peaks, we'll visualize them using the Integrative Genomics Viewer (IGV), which will allow us to take a look at how our reads pile up and contribute to the determination of peaks by the MACS2 algorithm.

<h3>Motif discovery</h3>

While peak calling can provide us with information on potential binding sites for a protein of interest, we can also perform further analyses to obtain information on specific binding motifs for a protein of interest. This process, called motif discovery, is used to identify similar patterns in peak sequences (<strong>Fig 4</strong>). These patterns can suggest that the enriched sequences are recognized by your protein of interest and can indicate a potential transcription factor binding motif.

<h4 style="text-align: center;"><strong>Fig 4</strong></h4>
<img src="./images/6_1_fig_4.png" style="width: 500px; margin: auto;"/>
<p style="text-align: center;">Image from: <a href="https://meme-suite.org/meme/doc/examples/meme_example_output_files/meme.html"><u>MEME Suite</u></a></p>

<h1 style="font-size: 40px; margin-bottom: 0px;">Data files for this module</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Since we didn't perform ChIP-seq on our MDA-MB-231 cells, we'll be making use of a publicly available dataset from <a href="#zanconatoetal2015"><u>Zonconato et al 2015 Nat Cell Biol</u></a>, where they performed ChIP-seq analyses on MDA-MB-231 cells to identify how the YAP/TAZ transcriptional response is mediated.

The data files for ChIP-seq can be found in the <code>shared</code> directory. This is a directory that is available to everyone on the Biology Hub, but cannot be modified. Inside the <code>shared</code> directory, you'll find a number of subdirectories. The directory path that we're specifically interested in is <code>~/shared/2025-fall/courses/1547808</code>. There, you'll find our <code>chip-seq</code> and <code>rna-seq</code> subdirectories that will contain the files you will need for ChIP-seq (and our RNA-seq files, too).

During class, we'll be doing a guided analysis using truncated sequence files that contain just 10 million reads (vs the almost 100 million reads of the full sequence file). These files have a prefix of <code>10M_</code> to indicate that this is the truncated sequence file. You will take what you've learned in class to perform the analysis on the full dataset.

<h1 style="font-size: 40px; margin-bottom: 0px;">Command line refresher</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Since a number of steps both for analyzing our ChIP-seq dataset and our RNA-seq dataset involve using shell commands, we'll refresh ourselves with some more Terminal (our command line interface). 

<h2>The command line</h2>

Recall from MCB201A 16-1, that when you start up Terminal, you'll be greeted with an empty prompt that looks something like:

<pre style="width: 350px; margin-top: 15px; margin-bottom: 15px; color: #000000; background-color: #EEEEEE; border: 1px solid; border-color: #AAAAAA; padding: 10px; border-radius: 15px; font-size: 12px;">jovyan@jupyter-CalNetID:~$</pre>

<h2>Some command line interface functionality</h2>

Small note that if you're following along running bash commands in this notebook, this functionality isn't available in the same way in your notebook.

<h3>Pull up a previous/subsequent line</h3>

If you want to pull up a previously executed command, you can press the <kbd>&uarr;</kbd> arrow key to bring up a previous command, and you can use the <kbd>&darr;</kbd> to go to the subsequent command. This can come in handy if you want to either re-run a previously executed command or if you want to bring up a previous line and make modificaitons to it, so you don't have to retype everything.

<h3>Terminate an actively running process</h3>

If you're running something in Terminal and you want to forcefully terminate the process, you can do so by pressing <kbd>Ctrl</kbd>+<kbd>C</kbd>, which will essentially abort the process that is actively running in Terminal.

<h3>Suspend an actively running process</h3>

In a more "gentler" way, you can also suspend an actively running process, which will not terminate it/abort its operation. You can do this by instead pressing <kbd>Ctrl</kbd>+<kbd>Z</kbd>. This will temporarily pause a program without shutting it down.

You can then restart the program back into the foreground by inputting <code>fg</code> into the command line and pressing <kbd>Enter</kbd> to execute the command.

<h3>Run a process in the background</h3>

To run a process in the background, you can either end the command line with an <code>&amp;</code>, which will indicate that you want the process to run in the background when the command is executed. This can be helpful for when you want to have the ability to use Terminal while some process runs in the background.

If you forget to input a <code>&amp;</code> at the end of the line, you can always press <kbd>Ctrl</kbd>+<kbd>Z</kbd>, which will pause the process. Then if you input <code>bg</code> and execute the command, the paused process will resume running in the background.

<h1 style="font-size: 40px; margin-bottom: 0px;">Command line syntax</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Since now we'll start using the command line to do more than just move around our Biology Hub, you'll need to understand the syntax for command line so that Terminal can understand and execute your input correctly and as you intended.

<h4 style="text-align: center;"><strong>Fig 5</strong></h4>
<img src="./images/6_1_fig_5.png" style="width: 300px; margin: auto;"/>

In a loose way, the organization of command line syntax isn't too different from Python. The <strong>prompt</strong>, while not present in Python, indicates the start of a command line. The <strong>command</strong> is like a function in Python, where it executes some pre-defined set of operations. The <strong>option</strong> and the <strong>argument</strong> can both be thought of as parameters for Python functions. Options alter how the command behaves, while arguments tell the command what it should operate on. OPtions are preceded by either a <code>&dash;</code> or <code>&dash;&dash;</code>, which indicate a short form (abbreviated) or long form option, respectively. Most commands have options that can be written in a short form or an equivalent long form, and you can find the different options for a command through its documentation.

The fact that there are sometimes two forms to an option, such as <code>&dash;h</code> and <code>&dash;&dash;help</code>, is kind of an artifact from when options used to have to be a single character.

In [2]:
%%bash

#Example of basic command
ls --help

Usage: ls [OPTION]... [FILE]...
List information about the FILEs (the current directory by default).
Sort entries alphabetically if none of -cftuvSUX nor --sort is specified.

Mandatory arguments to long options are mandatory for short options too.
  -a, --all                  do not ignore entries starting with .
  -A, --almost-all           do not list implied . and ..
      --author               with -l, print the author of each file
  -b, --escape               print C-style escapes for nongraphic characters
      --block-size=SIZE      with -l, scale sizes by SIZE when printing them;
                               e.g., '--block-size=M'; see SIZE format below
  -B, --ignore-backups       do not list implied entries ending with ~
  -c                         with -lt: sort by, and show, ctime (time of last
                               modification of file status information);
                               with -l: show ctime and sort by name;
                               othe

<h2>Spaces</h2>

Spaces in command line have meaning and are used to separate the different parts of an input. If you refer back to <strong>Fig 5</strong>, you can see that each part of an input is separated by a space, so the command an dindividual options and arguments are all separated from each other with a space. This means that you will want to be careful not to include spaces in your file name, otherwise it may appear as though you are providing additional arguments or options. If your file name does have spaces, you'll need to wrap it in single or double quotes, such as <code>"file name"</code>.

In [3]:
%%bash

#Example of handling spaces in command line
#Need to flank with single quote or double quotes
mkdir -p "my cat"

<h2>Line breaks</h2>

There will be times where we'll encounter long command line inputs, so we'll need to use line breaks to separate out the physical lines but still have them be interpreted as a single logical line by our command line interpreter. To introduce physical line breaks, you can input a backslash <code>&bsol;</code> and then pressing <kbd>Enter</kbd>. This will create a physical line break and allow you to continue inputting the next physical line without executing your command.

In [4]:
%%bash

mkdir \
-p \
liebchen

#Can separate lines in command line with line breaks indicated by a backslash
#You don't need to type in the preceding > that shows up in Terminal for each new line
#Can just type as shown here

In Terminal, you'll see that when you initiate a line break, there will be a <code>&gt;</code> at the beginning of each new physical line. This just indicates that the empty line continues from the preceding one.

<h2>Finding documentation</h2>

You can output the documentation for different commands by inputting <code>&dash;&dash;help</code> as an option following the command. The documentation usually provides details on how to use that command as well as possible options that you can input to modify how the command executes.

In [5]:
%%bash

#To pull up documentation, you can use -h or --help options depending on the command

ls --help

Usage: ls [OPTION]... [FILE]...
List information about the FILEs (the current directory by default).
Sort entries alphabetically if none of -cftuvSUX nor --sort is specified.

Mandatory arguments to long options are mandatory for short options too.
  -a, --all                  do not ignore entries starting with .
  -A, --almost-all           do not list implied . and ..
      --author               with -l, print the author of each file
  -b, --escape               print C-style escapes for nongraphic characters
      --block-size=SIZE      with -l, scale sizes by SIZE when printing them;
                               e.g., '--block-size=M'; see SIZE format below
  -B, --ignore-backups       do not list implied entries ending with ~
  -c                         with -lt: sort by, and show, ctime (time of last
                               modification of file status information);
                               with -l: show ctime and sort by name;
                               othe

<h2>Navigating in Terminal refresher</h2>

Below is just a quick refresher to navigate through directories and subdirectories using Terminal. For those of you following in your notebooks, you'll want to keep in mind that once your code cell finishes executing your bash command, it'll exit its instance of Terminal. So if you change directories, your current working directory will be reset back to what it was before when you run in a new code cell, as if you started up a new Terminal session.

In [8]:
%%bash

#Navigating through directories

#list contents
ls

#change directory to the specified one
cd cat

 
cat
fastqc-results
images
liebchen
liebchen_1.txt
liebchen_head.txt
liebchen meows
liebchen.txt
ls_help.txt
my
my cat
POSTCLASS_6-1_Intro_ChIP_seq.ipynb
week_6_homework


<ul>
    <li><code>ls</code> - this command lists the contents of your current working directory (or if you provide a path, the contents of the specified directory</li>
    <li><code>cd</code> - this command changes the current working directory to the specified directory</li>
    <li><code>mkdir</code> - this command lets you make directories in the current directory or if you provide a path, then in the specified path</li>
    <li><code>mv</code> - this command lets you move a file (or folder) to the specified path</li>
    <li><code>cp</code> - this command lets you copy a file or directory (using the <code>&dash;r</code> option) into the specified path</li>
    <li><code>rm</code> - this command lets you delete a file or directory (using the <code>&dash;r</code> option)</li>
</ul>

In [10]:
%%bash

#Run ls command again
ls

#Can see that the working directory is changed even though earlier cell changed to cat
#So each code cell is operating separately from others
echo "changing directory"
cd cat
echo "below this is cat directory contents"
ls

 
cat
fastqc-results
images
liebchen
liebchen_1.txt
liebchen_head.txt
liebchen meows
liebchen.txt
ls_help.txt
my
my cat
POSTCLASS_6-1_Intro_ChIP_seq.ipynb
week_6_homework
changing directory
below this is cat directory contents
liebchen.txt


<h1 style="font-size: 40px; margin-bottom: 0px;">Standard input/output streams</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

In the command line, there are standard streams for handling input and output(s), and in the command line/Terminal, the input fed into a command is referred to as the standard input <code>stdin</code>. Outputs from the command can go through two different standard streams, either the <code>stdout</code> or the <code>stderr</code>. The <code>stdout</code> is the standard output which handles the normal outputs from a command, and the <code>stderr</code> is the standard error, which as its name suggests is usually used to handle error outputs and diagnostic outputs from a command.

For example, the <code>echo</code> command will output into the <code>stdout</code>, which we can see in Terminal. This command functions more or less the same as the <code>print()</code> command that we've been using in Python, but for Terminal.

In [11]:
%%bash

#echo command acts basically like the print() function in Python
echo "MEOW!"
#echo will output to the stdout stream, which is what we see normally

MEOW!


We can also see the same using the <code>ls</code> command that we use to output file names. 

In [12]:
%%bash

#ls command also outputs to stdout
ls

#In general, there are two output streams that handle outputs
#Both will show up in Terminal

 
cat
fastqc-results
images
liebchen
liebchen_1.txt
liebchen_head.txt
liebchen meows
liebchen.txt
ls_help.txt
my
my cat
POSTCLASS_6-1_Intro_ChIP_seq.ipynb
week_6_homework


<h2>Redirecting an output</h2>

We can redirect the standard output streams by making use of the <code>&gt;</code> redirect symbol. If we follow this with an output file name, we can redirect to that specific file to store the output. In a way, this type of redirection can be thought of as using the <code>open()</code> + <code>file.write()</code> commands that we've used in Python with the <code>'w'</code> write mode for the <code>open()</code> function. So the result of redirecting an output with <code>&gt;</code> is that the output file is generated (if it doesn't exist), and the <code>stdout</code> is written to that file. 

In [13]:
%%bash

#You can control the output though via redirecting it
#The > indicates that you are redirecting the stdout of echo to the following file
echo "MEOW!" > liebchen_1.txt
#You should see now there's no output in Terminal
#It is instead redirected to the newly created file
#Note that > will overwrite anything if the file already exists

We also have the option to append outputs to an existing file, like how <code>open()</code> also has an <code>'a'</code> append mode. To do this in command line, we can use <code>&gt;&gt;</code> to redirect the <code>stdout</code> to append to the end of a file.

In [14]:
%%bash

#You can redirect to append to a file's existing contents as well
echo "MEOW! MEOW!" >> liebchen_1.txt
#Again, there will be no output in Terminal
#because it is being redirected to be appended to the indicated file

<h2>Piping</h2>

Piping is kind of a subset of redirecting an output, where you pass (or "pipe") the output of one command as an input for the next command. To do this, you'll use the <code>&vert;</code> pipe symbol, where the command to the left of the pipe is used as the input for the command to the right of the pipe.

For example, we can generate a list of unsorted files:

In [15]:
%%bash

#Use -f option for the ls command to generate a list of unsorted files
ls -f

liebchen meows
my
ls_help.txt
.
POSTCLASS_6-1_Intro_ChIP_seq.ipynb
week_6_homework
my cat
cat
liebchen
liebchen_head.txt
.gitkeep
fastqc-results
liebchen.txt
images
liebchen_1.txt
.ipynb_checkpoints
 
..


And then we can pipe that into a command to sort the output from the <code>ls</code> command.

In [16]:
%%bash

#Then we can pipe that into other things

ls -f | sort

 
.
..
cat
fastqc-results
.gitkeep
images
.ipynb_checkpoints
liebchen
liebchen_1.txt
liebchen_head.txt
liebchen meows
liebchen.txt
ls_help.txt
my
my cat
POSTCLASS_6-1_Intro_ChIP_seq.ipynb
week_6_homework


We can also combine this with the <code>&gt;</code> redirect symbol to then redirect the output to a new <code>.txt</code> file.

In [20]:
%%bash

ls -f | sort > sorted_files.txt

<h1 style="font-size: 40px; margin-bottom: 0px;">Opening files in command line</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

There are a few different ways of opening up files in command line to view the contents, and a common way to open up plain text files is by using the <code>cat</code> command, which simply concatenates the file into the <code>stdout</code> stream.

In [21]:
%%bash

cat liebchen_1.txt

MEOW!
MEOW! MEOW!


We can also use this to concatenate files together if we combined it with the <code>&gt;</code> redirect symbol.

In [22]:
%%bash

cat liebchen_1.txt sorted_files.txt > meow.txt

Similar to Python, you can also view just the first set of lines <code>head</code> or the last set of lines <code>tail</code>.

In [23]:
%%bash

head meow.txt

MEOW!
MEOW! MEOW!
 
.
..
cat
fastqc-results
.gitkeep
images
.ipynb_checkpoints


In [24]:
%%bash

tail meow.txt

liebchen_1.txt
liebchen_head.txt
liebchen meows
liebchen.txt
ls_help.txt
my
my cat
POSTCLASS_6-1_Intro_ChIP_seq.ipynb
sorted_files.txt
week_6_homework


By default, these commands will output 10 lines (from top or from bottom). But if we wanted to specify the number of lines to output, we can provide that to the command to modify how many lines we want it to output into the <code>stdout</code>.

In [26]:
%%bash

head -4 ls_help.txt

Usage: ls [OPTION]... [FILE]...
List information about the FILEs (the current directory by default).
Sort entries alphabetically if none of -cftuvSUX nor --sort is specified.



For those of you following along in Terminal, you can also open up your file to a temporary page viewer by using the <code>less</code> command. This command will pull up the contents of the indicated file, but it will not be part of your <code>stdout</code>. Instead, it'll be in a temporary viewer, which you can scroll through and also exit by pressing <kbd>Q</kbd>, and you'll return to an empty prompt. Note that this won't work in this notebook, and it'll be something you do in Terminal.

You can also pipe an output to a temporary viewer as well by piping into the <code>less</code> command.

In [27]:
%%bash

#Note that this should be run in Terminal since it doesn't work the same for notebooks
head -15 ls_help.txt | less

Usage: ls [OPTION]... [FILE]...
List information about the FILEs (the current directory by default).
Sort entries alphabetically if none of -cftuvSUX nor --sort is specified.

Mandatory arguments to long options are mandatory for short options too.
  -a, --all                  do not ignore entries starting with .
  -A, --almost-all           do not list implied . and ..
      --author               with -l, print the author of each file
  -b, --escape               print C-style escapes for nongraphic characters
      --block-size=SIZE      with -l, scale sizes by SIZE when printing them;
                               e.g., '--block-size=M'; see SIZE format below
  -B, --ignore-backups       do not list implied entries ending with ~
  -c                         with -lt: sort by, and show, ctime (time of last
                               modification of file status information);
                               with -l: show ctime and sort by name;


<h1 style="font-size: 40px; margin-bottom: 0px;">Exercise set</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

This is just a set of exercises for you to refresh yourself in Terminal and practice some new commands that you learned today just to refamiliarize yourself on working with command line.

<ol>
    <li>Return to your home directory</li>
    <li>Navigate to this week's directory</li>
    <li>Create a new folder named <code>fastqc-results</code> in this week's directory</li>
    <li>Navigate to <code>fastqc-results</code> and create a subdirectory named <code>seq-view</code></li>
    <li>Change directories to <code>seq-view</code></li>
    <li>Output the first 10 <u>reads</u> of <code>10M_ctrl_1.fastq</code> into <code>stdout</code></li>
    <li>Redirect the output to a new <code>.fastq</code> file in your <code>seq-view</code> folder</li>
    <li>Repeat #6-8 for <code>10M_taz_1.fastq</code></li>
</ol>

In [28]:
%%bash

#Change to home directory
cd ~
#Can also just be cd

#Navigate to this week's directory
cd ~/MCB201B_F2025/Week_6

#Create a new folder
mkdir -p fastqc-results
#You can add the -p option to avoid the error if file already exists

#Navigate into fastqc-results
cd fastqc-results

#Create a new folder
mkdir -p seq-view

#Navigate into seq-view
cd seq-view

#Output first 10 reads into stdout
head -40 ~/shared/2025-fall/courses/1547808/chip-seq/truncated/10M_ctrl_1.fastq

#Redirect instead to a file
head -40 ~/shared/2025-fall/courses/1547808/chip-seq/truncated/10M_ctrl_1.fastq > 10_reads_ctrl_1.fastq

#Repeat for taz_1
head -40 ~/shared/2025-fall/courses/1547808/chip-seq/truncated/10M_taz_1.fastq
head -40 ~/shared/2025-fall/courses/1547808/chip-seq/truncated/10M_taz_1.fastq > 10_reads_taz_1.fastq


@SRR1810912.1 HWI-ST1210:136:C1RBDACXX:5:1101:1472:2121 length=50
CACTGTAATTTCCCTAAAACAGGAGGTCACGGTAAGTCCTATGGATAGAT
+SRR1810912.1 HWI-ST1210:136:C1RBDACXX:5:1101:1472:2121 length=50
??????????????????????????????????????????????????
@SRR1810912.2 HWI-ST1210:136:C1RBDACXX:5:1101:1374:2127 length=50
CCCCGTGCTCCAGCCCAGCCAGGCCGCGCTGTCAGAAGGGATCTATCANN
+SRR1810912.2 HWI-ST1210:136:C1RBDACXX:5:1101:1374:2127 length=50
??????????????????????????????????????????????????
@SRR1810912.3 HWI-ST1210:136:C1RBDACXX:5:1101:1373:2149 length=50
ACAAAAAAACACAATACAATTTAAAAAGTATGAGTTATCTGAATATACAT
+SRR1810912.3 HWI-ST1210:136:C1RBDACXX:5:1101:1373:2149 length=50
??????????????????????????????????????????????????
@SRR1810912.4 HWI-ST1210:136:C1RBDACXX:5:1101:1298:2192 length=50
ACGGGATTATAAATCCCCATGTGCACTAAAGGGAAGGGTCTAGAAGGTCN
+SRR1810912.4 HWI-ST1210:136:C1RBDACXX:5:1101:1298:2192 length=50
??????????????????????????????????????????????????
@SRR1810912.5 HWI-ST1210:136:C1RBDACXX:5:1101:1327:2194 length=5

<h1 style="font-size: 40px; margin-bottom: 0px;">FastQC</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/" rel="noopener noreferrer" target="_blank"><u>FastQC</u></a> is developed by Simon Andrews at the Babraham Institute in the UK. FastQC is a quick way to take a look at the quality of your raw sequence reads in order to get a general idea of how it looks and if there may be a bias in your data that could affect your analysis and/or interpretation of your data. There is a range of analyses that FastQC performs to assess the quality of your raw sequence reads, and they help you to get an idea of whether an issue may have occurred during sample prep, library prep, or sequencing.

There are a number of analysis modules with more detailed information and examples that can be found on <a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/3%20Analysis%20Modules/" rel="noopener noreferrer" target="_blank"><u>the FastQC documentation pages.</u></a>

While it is helpful to run FastQC to get an idea of the quality of your sequence data, you should also be aware that each library prep will be different and just because it shows up as a <strong>Fail</strong> in FastQC, it may be fine in the context of what you're studying. The results of the FastQC file may point you to where you may want to look if it the outcome is not what you expect.

<h2>Running FastQC in Terminal</h2>

We'll do a guided practice to perform a FastQC analysis on truncated sequence files:

<ul>
    <li><code>10M_ctrl_1.fastq</code></li>
    <li><code>10M_taz_1.fastq</code></li>
</ul>

I recommend following along in Terminal, but you can also do this in the notebook as well, but you'll then need to have all your command lines in a single code cell.

In [29]:
%%bash

#Change to shared directory containing truncated files
cd ~/shared/2025-fall/courses/1547808/chip-seq/truncated

#Running fastqc
fastqc -o ~/MCB201B_F2025/Week_6/fastqc-results 10M_ctrl_1.fastq

Started analysis of 10M_ctrl_1.fastq
Approx 5% complete for 10M_ctrl_1.fastq
Approx 10% complete for 10M_ctrl_1.fastq
Approx 15% complete for 10M_ctrl_1.fastq
Approx 20% complete for 10M_ctrl_1.fastq
Approx 25% complete for 10M_ctrl_1.fastq
Approx 30% complete for 10M_ctrl_1.fastq
Approx 35% complete for 10M_ctrl_1.fastq
Approx 40% complete for 10M_ctrl_1.fastq
Approx 45% complete for 10M_ctrl_1.fastq
Approx 50% complete for 10M_ctrl_1.fastq
Approx 55% complete for 10M_ctrl_1.fastq
Approx 60% complete for 10M_ctrl_1.fastq
Approx 65% complete for 10M_ctrl_1.fastq
Approx 70% complete for 10M_ctrl_1.fastq
Approx 75% complete for 10M_ctrl_1.fastq
Approx 80% complete for 10M_ctrl_1.fastq
Approx 85% complete for 10M_ctrl_1.fastq
Approx 90% complete for 10M_ctrl_1.fastq
Approx 95% complete for 10M_ctrl_1.fastq
Approx 100% complete for 10M_ctrl_1.fastq


Analysis complete for 10M_ctrl_1.fastq


<h2>Let's breakdown the code:</h2>

<code>fastqc</code>

This is our command. We are instructing Terminal to run the fastqc program.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>-o</code>

This is an option that we input to modify how the command behaves. In this case, we are instructing it to output the resulting files to a specific path that we provide after this option.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>~/MCB201B_F2025/Week_6/fastqc-results</code>

This is the path to the directory that we want the fastqc results to be saved. In other words, this is the output directory.

<hr style="border: 1px solid; border-color: #BBBBBB;"></hr>

<code>10M_ctrl_1.fastq</code>

This is the <code>.fastq</code> file that we want the fastqc program to analyze. If our current working directory did not have this file name, then we would also have to include the path to the <code>.fastq</code> file.

If we want to very crudely rewrite/'translate' this to the Python syntax that we're familiar with, we can kind of see what we're doing in command line isnt too different than calling up a function in Python and providing it with specific arguments so that it knows what to do:

```
fastqc(o='~/MCB201B_F2025/Week_6/fastqc-results',
       input='10M_ctrl_1.fastq'
       )
```
Note that this isn't actually something you can call up in Python, it's just for example.

<h1 style="font-size: 40px; margin-bottom: 0px;">FastQC exercise set</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

Now give it a try for the other ChIP-seq <code>.fastq</code> files.

<ol>
    <li>Run <code>fastqc</code> for the other truncated ChIP-seq files</li>
    <li>Once you're done with the analyses, use the File Browser to open up the HTML output files</li>
    <li>Take a look at the analysis results</li>
    <li>How does the quality of the sequencing data look like for these samples?</li>
    <li>If you have extra time in class, you can feel free to get started on running the fastqc analysis on the full datasets (this will be part of this week's assignment)</li>
</ol>

In [30]:
%%bash

#Change to shared directory containing truncated files
cd ~/shared/2025-fall/courses/1547808/chip-seq/truncated

#Running fastqc
fastqc -o ~/MCB201B_F2025/Week_6/fastqc-results 10M_ctrl_2.fastq
fastqc -o ~/MCB201B_F2025/Week_6/fastqc-results 10M_ctrl_3.fastq
fastqc -o ~/MCB201B_F2025/Week_6/fastqc-results 10M_taz_1.fastq
fastqc -o ~/MCB201B_F2025/Week_6/fastqc-results 10M_taz_2.fastq

#Later on we'll learn to script in bash, but for now, we'll just get used to working in command line
#Running one command at a time

Started analysis of 10M_ctrl_2.fastq
Approx 5% complete for 10M_ctrl_2.fastq
Approx 10% complete for 10M_ctrl_2.fastq
Approx 15% complete for 10M_ctrl_2.fastq
Approx 20% complete for 10M_ctrl_2.fastq
Approx 25% complete for 10M_ctrl_2.fastq
Approx 30% complete for 10M_ctrl_2.fastq
Approx 35% complete for 10M_ctrl_2.fastq
Approx 40% complete for 10M_ctrl_2.fastq
Approx 45% complete for 10M_ctrl_2.fastq
Approx 50% complete for 10M_ctrl_2.fastq
Approx 55% complete for 10M_ctrl_2.fastq
Approx 60% complete for 10M_ctrl_2.fastq
Approx 65% complete for 10M_ctrl_2.fastq
Approx 70% complete for 10M_ctrl_2.fastq
Approx 75% complete for 10M_ctrl_2.fastq
Approx 80% complete for 10M_ctrl_2.fastq
Approx 85% complete for 10M_ctrl_2.fastq
Approx 90% complete for 10M_ctrl_2.fastq
Approx 95% complete for 10M_ctrl_2.fastq
Approx 100% complete for 10M_ctrl_2.fastq


Analysis complete for 10M_ctrl_2.fastq


Started analysis of 10M_ctrl_3.fastq
Approx 5% complete for 10M_ctrl_3.fastq
Approx 10% complete for 10M_ctrl_3.fastq
Approx 15% complete for 10M_ctrl_3.fastq
Approx 20% complete for 10M_ctrl_3.fastq
Approx 25% complete for 10M_ctrl_3.fastq
Approx 30% complete for 10M_ctrl_3.fastq
Approx 35% complete for 10M_ctrl_3.fastq
Approx 40% complete for 10M_ctrl_3.fastq
Approx 45% complete for 10M_ctrl_3.fastq
Approx 50% complete for 10M_ctrl_3.fastq
Approx 55% complete for 10M_ctrl_3.fastq
Approx 60% complete for 10M_ctrl_3.fastq
Approx 65% complete for 10M_ctrl_3.fastq
Approx 70% complete for 10M_ctrl_3.fastq
Approx 75% complete for 10M_ctrl_3.fastq
Approx 80% complete for 10M_ctrl_3.fastq
Approx 85% complete for 10M_ctrl_3.fastq
Approx 90% complete for 10M_ctrl_3.fastq
Approx 95% complete for 10M_ctrl_3.fastq
Approx 100% complete for 10M_ctrl_3.fastq


Analysis complete for 10M_ctrl_3.fastq


Started analysis of 10M_taz_1.fastq
Approx 5% complete for 10M_taz_1.fastq
Approx 10% complete for 10M_taz_1.fastq
Approx 15% complete for 10M_taz_1.fastq
Approx 20% complete for 10M_taz_1.fastq
Approx 25% complete for 10M_taz_1.fastq
Approx 30% complete for 10M_taz_1.fastq
Approx 35% complete for 10M_taz_1.fastq
Approx 40% complete for 10M_taz_1.fastq
Approx 45% complete for 10M_taz_1.fastq
Approx 50% complete for 10M_taz_1.fastq
Approx 55% complete for 10M_taz_1.fastq
Approx 60% complete for 10M_taz_1.fastq
Approx 65% complete for 10M_taz_1.fastq
Approx 70% complete for 10M_taz_1.fastq
Approx 75% complete for 10M_taz_1.fastq
Approx 80% complete for 10M_taz_1.fastq
Approx 85% complete for 10M_taz_1.fastq
Approx 90% complete for 10M_taz_1.fastq
Approx 95% complete for 10M_taz_1.fastq
Approx 100% complete for 10M_taz_1.fastq


Analysis complete for 10M_taz_1.fastq


Started analysis of 10M_taz_2.fastq
Approx 5% complete for 10M_taz_2.fastq
Approx 10% complete for 10M_taz_2.fastq
Approx 15% complete for 10M_taz_2.fastq
Approx 20% complete for 10M_taz_2.fastq
Approx 25% complete for 10M_taz_2.fastq
Approx 30% complete for 10M_taz_2.fastq
Approx 35% complete for 10M_taz_2.fastq
Approx 40% complete for 10M_taz_2.fastq
Approx 45% complete for 10M_taz_2.fastq
Approx 50% complete for 10M_taz_2.fastq
Approx 55% complete for 10M_taz_2.fastq
Approx 60% complete for 10M_taz_2.fastq
Approx 65% complete for 10M_taz_2.fastq
Approx 70% complete for 10M_taz_2.fastq
Approx 75% complete for 10M_taz_2.fastq
Approx 80% complete for 10M_taz_2.fastq
Approx 85% complete for 10M_taz_2.fastq
Approx 90% complete for 10M_taz_2.fastq
Approx 95% complete for 10M_taz_2.fastq
Approx 100% complete for 10M_taz_2.fastq


Analysis complete for 10M_taz_2.fastq


<h1 style="font-size: 40px; margin-bottom: 0px;">Interpreting FastQC results</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

After running <code>fastqc</code>, you'll have two output files, one of which is an HTML file and the other which is a .zip file containing the individual graphs as separate files, as well as the data for each of those graphs.

<a href="https://www.bioinformatics.babraham.ac.uk/projects/fastqc/" rel="noopener noreferrer" target="_blank"><u>Documentation for FastQC can be found here.</u></a>

<h2>Basic statistics</h2>

The first thing you'll see is the summary statistics and general file information for your sequence file (<strong>Fig 6</strong>). This provides you general information about your sequence file.

<h4 style="text-align: center;"><strong>Fig 6</strong></h4>
<img src="./images/6_1_fig_6.png" style="height: 250px; margin: auto;"/>
<p style="text-align: center;">Summary statistics for <code>10M_ctrl_1.fastq</code></p>

<h2>Per base sequence quality</h2>

<h4 style="text-align: center;"><strong>Fig 7</strong></h4>
<img src="./images/6_1_fig_7.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per base sequence quality for <code>10M_ctrl_1.fastq</code></p>

Next, you'll see the per base sequence quality, which displays the stastitical information on your sequence read quality (<strong>Fig 2</strong>). The X-axis is the position along the read, while the Y-axis is the average quality score at that specific position for all reads. Recall from lecture and lesson 6.1 that .fastq files also contain the read quality information for each base of a sequence read, and the information is usually conveyed through a Phred score:

```
@SRR1810912.1 HWI-ST1210:136:C1RBDACXX:5:1101:1472:2121 length=50
CACTGTAATTTCCCTAAAACAGGAGGTCACGGTAAGTCCTATGGATAGAT
+SRR1810912.1 HWI-ST1210:136:C1RBDACXX:5:1101:1472:2121 length=50
??????????????????????????????????????????????????
```

This module takes the average score at each position for all sequences and visualizes the data in the form of a box-and-whisker plot, with the inner quartile range covered by the yellow box and the whiskers depicting the lower and upper 10th percentiles. The blue line represents the calculated average Phred score at that postion, and in the case of the ChIP-seq data we're using, all the bases appear to have a Phred-33 score of 30, represented by the ASCII <code>&quest;</code>.

The results of this analysis module can be quite important in that it lets you know if there may have been something that went wrong during sequencing, especially if most of the bases are of poor or highly variable quality. Usually, towards the end of a sequence read, the quality should dip a bit because of the nature of sequencing. Somtimes the elongation fails to continue in a greater and greater proportion of the strands or the clonal strands desync, so then the base can't be accurately called from the mixed fluorescent signal.

<h2>Per tile sequence quality</h2>

<h4 style="text-align: center;"><strong>Fig 8</strong></h4>
<img src="./images/6_1_fig_8.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per tile sequence quality for <code>10M_ctrl_1.fastq</code></p>

For Illumina sequencing platforms, each sequence read retains its original sequence identifier, so the read can be traced back to its specific flow cell tile (<strong>Fig 9</strong>).

<h4 style="text-align: center;"><strong>Fig 9</strong></h4>
<img src="./images/6_1_fig_9.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Image from Whiteford et al 2009</p>

The Illumina sequencing platform makes use of flow cells, which contain nanoscale wells that allow for clonal library amplification and subsequent sequencing by synthesis. If you recall from MCB201A and Dr. Ingolia's lectures, Illumina sequencing by synthesis technology uses fluorescently labeled nucleotides whose signal can be captured via imaging. Repeated images of the same section (a tile) of a flowcell allow for the sequence of the clonally amplified fragments to be read out as the progression of fluorescence signals when the clonal population of DNA fragments is extended. A tile is then considered a defined section of the flow cell that is repeatedly imaged, and a single tile can capture the sequences of a large number of clonal fragments (or clusters). Within a single flow cell, there are many, many tiles.

So the information presented by this module can be thought of as a 3D matrix, where the 1st axis (or X-axis) contains the position of that base within the sequence reads, the 2nd axis (or Y-axis) contains essentially the tile ID, and the 3rd axis (the heatmap color) is the average read quality at that position (Phred score) for that tile.

Issues that are identified through this analysis module generally point to some issue with the sequencing run, such as an issue calling bases or something obscuring the view of the imager, which would lead to an inability to call the base with high confidence. Since the data we use for ChIP-seq has a Phred-33 score of <code>&quest;</code> for all bases, we see a homogeneous graph for the per tile sequence quality.

If we take a quick look at one sequence file from our RNA-seq data in <strong>Fig 10</strong>:

<h4 style="text-align: center;"><strong>Fig 10</strong></h4>
<img src="./images/6_1_fig_10.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per tile sequence quality for <code>2024 RNA-seq data</code></p>

We can see that most of the tiles look good, but for two of the tiles, there is an issue with the read quality in the later positions of the sequence reads. This type of issue generally points to a potential obstruction of the flow cell that occurred during imaging, which occurred partway through sequencing, so the later sequence read quality is impacted. This could be due to something like dust or some other particulate falling onto the flow cell or potentially something getting stuck in the flow cell, both of which can obscure those tiles and their fluorescence signals.

<h2>Per sequence quality score</h2>

<h4 style="text-align: center;"><strong>Fig 11</strong></h4>
<img src="./images/6_1_fig_11.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per sequence quality score for <code>10M_ctrl_1.fastq</code></p>

The next analysis module is the per sequence quality score (<strong>Fig 11</strong>), which plots the distribution of the different Phred scores among all your sequence reads. So it's essentially like a histogram, where the X-axis is the average Phred score for a sequence read (not position), and the Y-axis is the number of reads that have that mean Phred score. A good sequencing run will have the peak of your distribution towards the higher end of the mean Phred score, indicating that your reads are mostly all good quality.

Again, since the reads for the ChIP-seq dataset is all <code>&quest;</code>, we can get a better feel for this analysis module by looking at our RNA-seq dataset in <strong>Fig 12</strong>:

<h4 style="text-align: center;"><strong>Fig 12</strong></h4>
<img src="./images/6_1_fig_12.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per sequence quality score for <code>2024 RNA-seq data</code></p>

We can see that there is a nice peak towards the higher Phred scores, indicating that the vast majority of our reads are of good quality.

<h2>Per base sequence content</h2>

<h4 style="text-align: center;"><strong>Fig 13</strong></h4>
<img src="./images/6_1_fig_13.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per base sequence content for <code>10M_ctrl_1.fastq</code></p>

This analysis module takes a look at the proportions of each nucleotide for your sequence reads (<strong>Fig 13</strong>). Since our data comes from amplified fragments of the human genome, we would expect that the proportions of nucleotides should be close to the human genome. Since the GC content of the human genome is 41% (Bionumbers), we would expect both the G and C bases to comprise ~20% of the genome each, and both A and T to make up ~30% of the genome each. Over the millions of reads, the proportions to be similar to that of the human genome at each position of a read, so the four lines should be roughly parallel to one another.

Deviations greater than 10% trigger a warning, while those greater than 20% will trigger a fail. This may point to some bias in your library leading to an over- or under-representation of certain bases. In the case of this ChIP-seq dataset, the complete drop in the proportion of A nucleotides at the last position suggest that there may have been trimming of this dataset to remove Illumina adapters from the sequence read since the Illumina universal adapter starts with an A. The homogeneous read quality also suggest that it may have been trimmed.

You'll see later on with our RNA-seq data that it is usually expected to fail this analysis module.

<h2>Per sequence GC content</h2>

<h4 style="text-align: center;"><strong>Fig 14</strong></h4>
<img src="./images/6_1_fig_14.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per sequence GC content for <code>10M_ctrl_1.fastq</code></p>

While the per base sequence content looks at the proportions of each base at each position in a read, the per sequence GC content takes a look at the average GC content for all your sequence reads (<strong>Fig 14</strong>). In this analysis module, you're provided with the distribution of %GC content for your sequence file, and it gives you an idea if there may be an issue with your library prep. Usually, you should expect that without any experimental biases, the peak of your %GC content distribution should be match up roughly to the %GC content of your model organism's genome. The theoretical curve is calculated based on the sequence data you provide to FastQC, where it assumes a normal distribution around the modal GC content. If you see an unexpected distribution of %GC content, then that may point to an issue with your library, potentially an overexpressed gene or some contamination of your prep.

<h2>Per base N content</h2>

<h4 style="text-align: center;"><strong>Fig 15</strong></h4>
<img src="./images/6_1_fig_15.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Per base N content for <code>10M_ctrl_1.fastq</code></p>

This analysis module will let you know if there may have been issues calling bases along a specific position among all your sequence reads (<strong>Fig 15</strong>). In the ChIP-seq dataset that we're using, you can see that towards the end of the read, there is a fairly large increase in the proportion of Ns, causing this analysis module to be flagged as Fail. This usually points to an issue with the sequencing run.

Taken into consideration with the per base sequence content, there may have been an issue towards the end of the sequencing run.

<h2>Sequence length distribution</h2>

<h4 style="text-align: center;"><strong>Fig 16</strong></h4>
<img src="./images/6_1_fig_16.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Sequence length distribution for <code>10M_ctrl_1.fastq</code></p>

This plots the distribution of the lengths of your sequence reads (<strong>Fig 16</strong>). Since Illumina sequencing platforms create reads of all the same length, you should expect a single length for all your sequence reads. Here, we can see that all the sequences are 50bp in length.

Other sequencing platforms may have a range of sequence read lengths, and you would be able to see there distribution through this analysis module.

<h2>Sequence duplication levels</h2>

<h4 style="text-align: center;"><strong>Fig 17</strong></h4>
<img src="./images/6_1_fig_17.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Sequence duplication levels for <code>10M_ctrl_1.fastq</code></p>

The sequence duplication levels module plots the distribution of the number of duplicate sequences within your sequence file (<strong>Fig 17</strong>). This analysis module assumes that an ideal library will have low levels of a diverse range of different sequence reads, indicating broad coverage. So a high level of duplication could suggest that there may be an issue with your library prep, resulting in some bias and an overrepresentation of certain fragments/sequences. However, your experimental design can also influence the composition of your library, and sometimes, you may expect there to be high levels of duplication of certain sequences, such as for very highly expressed genes. Another potential reason why this module may get flagged is if you have saturated your sequencing run, so subsequent reads are no longer as unique.

<h2>Overrepresented sequences</h2>

<h4 style="text-align: center;"><strong>Fig 18</strong></h4>
<img src="./images/6_1_fig_18.png" style="height: 50px; margin: auto;"/>
<p style="text-align: center;">Overrepresented sequences for <code>10M_ctrl_1.fastq</code></p>

This module takes a look at the sequnces that appear to be overrepresented in your sample (<strong>Fig 18</strong>), and it can point to potential adapter contamination or some other bias in your dataset.

<h2>Adapter content</h2>

<h4 style="text-align: center;"><strong>Fig 19</strong></h4>
<img src="./images/6_1_fig_19.png" style="height: 350px; margin: auto;"/>
<p style="text-align: center;">Adapter content for <code>10M_ctrl_1.fastq</code></p>

Illumina sequencing requires adapters to be ligated to both ends of DNA fragments to be sequenced in order for the DNA fragment to be anchored to the flow cell and for it to be clonally amplified. These adapters also are needed for the sequencing primers to bind to the single strand DNA template for sequencing by synthesis to occur. This analysis module takes a look at your sequence reads to determine if there is potential adapter sequences within them (<strong>Fig 19</strong>). An unusually high amount of adapter sequences that were captured in your sequencing run would be flagged by this  module and would indicate that you may want to perform some adapter trimming prior to aligning your reads.

In the case of the ChIP-seq dataset that we're working with, it looks like there's no adapter contamination.

<h1 style="font-size: 40px; margin-bottom: 0px;">References</h1>

<hr style="margin-left: 0px; border: 0.25px solid; border-color: #000000; width: 98%;"></hr>

<p style="padding-left: 20px;"><a href="https://www.nature.com/articles/nrg2641" rel="noopener noreferrer" target="_blank"><u>Park 2009 Nat Rev Genet:</u></a> ChIP–seq: advantages and challenges of a maturing technology</p>

<p style="padding-left: 20px;"><a href="https://www.nature.com/articles/ncb3216" rel="noopener noreferrer" target="_blank"><u>Zanconato et al 2015 Nat Cell Biol:</u></a> ChIP–seq: advantages and challenges of a maturing technology</p>

<p style="padding-left: 20px;"><a href="https://www.nature.com/articles/ncb3216" rel="noopener noreferrer" target="_blank"><u>Wilbanks and Facciotti 2010 PLoS ONE:</u></a> Evaluation of Algorithm Performance in ChIP-Seq Peak Detection</p>

<p style="padding-left: 20px;"><a href="https://pubmed.ncbi.nlm.nih.gov/19549630/" rel="noopener noreferrer" target="_blank"><u>Whiteford et al 2009 Bioinformatics:</u></a> Swift: primary data analysis for the Illumina Solexa sequencing platform</p>

<p style="padding-left: 20px;"><a href="https://bionumbers.hms.harvard.edu/bionumber.aspx?id=100679&ver=5&trm=gc+content+human+genome&org=" rel="noopener noreferrer" target="_blank"><u>Bionumbers for human genome GC content</u></a></p>